In [1]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
from evaluate import error
from mmfi import make_dataset, make_dataloader

### Loda Data

In [2]:
dataset_root = 'd:\Data\My_MMFi_Data\MMFi_Dataset'
# xian zai shi yong de shi Radar_Fused
with open('config_copy_a.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

S02 ['A01', 'A02', 'A03', 'A04', 'A06', 'A08', 'A09', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A26']
S03 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A17', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S05 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A19', 'A20', 'A21', 'A24', 'A25']
S06 ['A01', 'A03', 'A04', 'A06', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S08 ['A01', 'A02', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A13', 'A14', 'A17', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S09 ['A01', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A17', 'A18', 'A20', 'A21', 'A24', 'A25', 'A27']
S11 ['A01', 'A02', 'A03', 'A05', 'A06', 'A08', 'A10', 'A11', 'A12', 'A13', 'A16', 'A17', 'A18', 'A19', 'A21', 'A23',

In [3]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.

    dict_keys(['modality', 'scene', 'subject', 'action', 'idx', 'output', 
    'input_rgb', 'input_depth', 'input_lidar', 'input_mmwave'])
    '''
    ## get sequence lengths
    # for t in batch:
    #     print(t.keys())
    #     print(a)
    # #     # print(t[0].shape,t[1].shape)
    # kpts = []
    # [kpts.append(np.array(t['output'])) for t in batch]
    # kpts = torch.FloatTensor(np.array(kpts))

    # lengths = torch.tensor([t['input_rgb'].shape[0] for t in batch ])
    all_actions = {'A01': 0., 'A02': 1., 'A03': 2., 'A04': 3., 'A05': 4., 
                'A06': 5., 'A07': 6., 'A08': 7., 'A09': 8., 'A10': 9.,
                'A11': 10., 'A12': 11., 'A13': 12., 'A14': 13., 'A15': 14., 
                'A16': 15., 'A17': 16., 'A18': 17., 'A19': 18., 'A20': 19., 
                'A21': 20., 'A22': 21., 'A23': 22., 'A24': 23., 'A25': 24., 
                'A26': 25., 'A27': 26.}
    
    labels = []
    [labels.append(all_actions[t['action']]) for t in batch]
    labels = torch.FloatTensor(labels)

    # rgb
    # rgb_data = np.array([(t['input_rgb']) for t in batch ])
    # rgb_data = torch.FloatTensor(rgb_data).permute(0,3,1,2)

    # # depth
    # depth_data = np.array([(t['input_depth']) for t in batch ])
    # depth_data = torch.FloatTensor(depth_data).permute(0,3,1,2)

    # # mmwave
    # ## padd
    # mmwave_data = [torch.Tensor(t['input_mmwave']) for t in batch ]
    # mmwave_data = torch.nn.utils.rnn.pad_sequence(mmwave_data)
    # ## compute mask
    # mmwave_data = mmwave_data.permute(1,0,2)

    # # lidar
    ## padd
    lidar_data = [torch.Tensor(t['input_lidar']) for t in batch ]
    lidar_data = torch.nn.utils.rnn.pad_sequence(lidar_data)
    ## compute mask
    lidar_data = lidar_data.permute(1,0,2)

    # # wifi-csi
    # wifi_data = np.array([(t['input_wifi-csi']) for t in batch ])
    # wifi_data = torch.FloatTensor(wifi_data)

    # return rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, lengths
    return lidar_data, labels

In [4]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)

In [5]:
for i, data in enumerate(train_loader):
    # rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, lengths = data
    # print(rgb_data[0].shape, depth_data[0].shape, lidar_data[0].shape, mmwave_data[0].shape, wifi_data[0].shape,kpts.shape, lengths.shape)
    # print(rgb_data.shape, depth_data.shape, lidar_data.shape, mmwave_data.shape, wifi_data.shape, kpts.shape, lengths.shape)
    lidar_data, label = data
    print(lidar_data.shape, label.shape)
    break

torch.Size([16, 1405, 3]) torch.Size([16])


C:\Users\CHEN_X~1\AppData\Local\Temp/ipykernel_2636/1602073218.py:49: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  lidar_data = [torch.Tensor(t['input_lidar']) for t in batch ]


### Model

In [6]:
from HAR_lidar_benchmark.lidar_point_transformer import *

model = lidar_PointTransformerReg()

# out = model(lidar_data)
# print(out.shape)

### training 

In [7]:
def test(model, tensor_loader, criterion, device):
    model.eval()
    test_acc = 0
    test_loss = 0
    for data in tqdm(tensor_loader):
        rgb_data, labels = data
        inputs = rgb_data.to(device)
        labels.to(device)
        labels = labels.type(torch.LongTensor)
        outputs = model(inputs)
        outputs = outputs.type(torch.FloatTensor)
        outputs.to(device)
        loss = criterion(outputs,labels)
        predict_y = torch.argmax(outputs,dim=1).to(device)
        accuracy = (predict_y == labels.to(device)).sum().item()
        test_acc += accuracy
        test_loss += loss.item() * labels.size(0)
        outputs = outputs.detach().numpy()
        labels = labels.detach().numpy()
    test_acc = test_acc/len(tensor_loader.dataset)
    test_loss = test_loss/len(tensor_loader.dataset)
    print("validation accuracy:{:.4f}, loss:{:.5f}".format(float(test_acc),float(test_loss)))
    return test_acc

In [12]:
def train(model, train_loader, test_loader, num_epochs, learning_rate, criterion, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
    # optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[20,40],gamma=0.1)
    parameter_dir = './HAR_lidar_benchmark/lidar_all_random.pt'
    best_test_acc = 0
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_accuracy = 0.7333
        num_iter = 2000
        for i, data in enumerate(tqdm(train_loader)):
            if i < num_iter:
                rgb_data, labels = data
                inputs = rgb_data.to(device)
                labels.to(device)
                labels = labels.type(torch.LongTensor)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                outputs = outputs.type(torch.FloatTensor)
                outputs.to(device)
                loss = criterion(outputs,labels)
                loss.backward()
                # print(length)
                # print("loss is ", loss.item())
                optimizer.step()
                
                epoch_loss += loss.item() * labels.size(0)
                predict_y = torch.argmax(outputs,dim=1).to(device)
                epoch_accuracy += (predict_y == labels.to(device)).sum().item()
                outputs = outputs.detach().numpy()
                labels = labels.detach().numpy()
            else:
                break
            # print("epoch loss is ", epoch_loss)
        epoch_loss = epoch_loss/(rgb_data.size(0)*num_iter)
        epoch_accuracy = epoch_accuracy/(rgb_data.size(0)*num_iter)
        print('Epoch:{}, Accuracy:{:.4f},Loss:{:.9f}'.format(epoch+1, float(epoch_accuracy),float(epoch_loss)))
        if (epoch+1) % 5 == 0:
            test_acc = test(
                model=model,
                tensor_loader=test_loader,
                criterion = criterion,
                device= device
            )
            if test_acc >= best_test_acc:
                print(f"best test acuracy is:{test_acc}")
                best_test_acc = test_acc
                torch.save(model.state_dict(), parameter_dir)
        # scheduler.step()
    return

In [13]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) 

cuda:0


In [14]:
criteria = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('./HAR_lidar_benchmark/lidar_all_random.pt'))
model.to(device)
train(
    model=model, 
    train_loader= train_loader,
    test_loader= val_loader,    
    num_epochs= 50,
    learning_rate=1e-4,
    criterion = criteria,
    device=device 
    )

 12%|█▏        | 2000/16038 [16:52<1:58:27,  1.97it/s]


Epoch:1, Accuracy:0.7657,Loss:0.711068300


 12%|█▏        | 2000/16038 [17:01<1:59:30,  1.96it/s]


Epoch:2, Accuracy:0.7853,Loss:0.646752351


 12%|█▏        | 2000/16038 [16:11<1:53:38,  2.06it/s]


...

Epoch:25, Accuracy:0.9318,Loss:0.210690901


100%|██████████| 4010/4010 [12:11<00:00,  5.48it/s]   


validation accuracy:0.7478, loss:0.97368


  0%|          | 9/16038 [00:03<1:45:47,  2.53it/s]


KeyboardInterrupt: 

In [8]:
criteria = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('./HAR_lidar_benchmark/lidar_all_random.pt'))
model.to(device)
test(
        model=model,
        tensor_loader=val_loader,
        criterion = criteria,
        device= device
    )

100%|██████████| 4010/4010 [10:04<00:00,  6.63it/s]

validation accuracy:0.7845, loss:0.75000


0.7845429604688864

In [11]:
parameter_dir = './HAR_lidar_benchmark/lidar_all_random.pt'
torch.save(model.state_dict(), parameter_dir)